In [124]:
import sys
import tensorflow as tf 
import numpy as np

from skimage.restoration import denoise_nl_means, estimate_sigma
from sklearn import metrics
from sklearn.metrics import classification_report

### TO RUN ON KAGGLE, RUN THE NEXT CELL TO IMPORT ALL THE NECESSARY CODE

In [95]:
# Clone the repository
!git clone https://github.com/jpscardoso97/ich-detection.git 

repo_name = "ich-detection"

sys.path.append(repo_name)

# Change to the repository directory
%cd {repo_name}

%pip install -r requirements.txt

from scripts.model import VGG

fatal: destination path 'ich-detection' already exists and is not an empty directory.
/kaggle/working/ich-detection
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 91.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 93.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 92.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 92.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: pywavelets
    Found existing installation: PyWavelets 1.5.0
    Uninstalling PyWavelets-1.5.0:
      Successfully uninstalled PyWavelets-1.5.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dty

## Define noise level for experiment

In [96]:
NOISE_LEVEL = 0.25
TEST_NOISE_LEVELS = [0, 0.25, 0.5, 0.75, 1, 1.5]

### Denoiser

In [117]:
def denoise_gaussian_image(image):
    sigma_est = estimate_sigma(image, average_sigmas=True, channel_axis=-1)
    denoised_image = denoise_nl_means(image, h= 9 * sigma_est, fast_mode=True, patch_size=5, patch_distance=3, channel_axis=-1)
    input_arr_3d = np.expand_dims(denoised_image, axis=-1)
    return input_arr_3d
    

## Load Data

In [118]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=denoise_gaussian_image)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=denoise_gaussian_image)

n = str.replace(str(NOISE_LEVEL), '.', '_')
train_file = '/kaggle/input/rsna-bme548-png/RSNA_BME548-sino-png/train_noisy_' + n
test_path = '/kaggle/input/rsna-bme548-png/RSNA_BME548-sino-png/test_noisy_'

# Generators
train_generator = train_datagen.flow_from_directory(
    train_file,
    target_size=(224, 224),
    batch_size=4,
    class_mode='binary',
    color_mode="grayscale",
    shuffle=False,
    seed=42
)

validation_generator = val_datagen.flow_from_directory(
    test_path + n,
    target_size=(224, 224),
    batch_size=4,
    class_mode='binary',
    color_mode="grayscale",
    shuffle=False,
    seed=42
)

Found 3199 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


### Train model

In [119]:
def train_model():
  optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.00002,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False
  )

  early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
  )

  model = VGG()
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    callbacks=[early_stopping]
  )

  return model

### Get predictions

In [120]:
def get_predictions(model, data_gen):
    preds = model.predict(data_gen)
    preds_binary = (preds > 0.5).astype(int)

    return preds_binary

### Classification Report

In [121]:
def get_classification_report(y_test, y_pred):
  accuracy = metrics.accuracy_score(y_test,y_pred)

  print("Accuracy of model=",accuracy)

  return classification_report(y_test, y_pred)

### Run experiments

In [125]:
import warnings
warnings.filterwarnings('ignore')

model = train_model()

Epoch 1/50


2024-05-02 17:20:02.083743: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.30583, expected 0.724921
2024-05-02 17:20:02.083809: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 1.7165, expected 1.13559
2024-05-02 17:20:02.083819: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.11479, expected 1.53388
2024-05-02 17:20:02.083826: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 1.91558, expected 1.33467
2024-05-02 17:20:02.083834: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 1.79034, expected 1.20943
2024-05-02 17:20:02.083841: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 2.01195, expected 1.43103
2024-05-02 17:20:02.083849: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 1.82977, expected 1.24886
2024-05-02 17:20:02.083856: E external/local_xla/xla/se

765/800 ━━━━━━━━━━━━━━━━━━━━ 4s 116ms/step - accuracy: 0.5934 - loss: 0.9120

2024-05-02 17:21:34.801345: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.37517, expected 0.381106
2024-05-02 17:21:34.801409: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 1.39267, expected 0.398603
2024-05-02 17:21:34.801423: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 1.75861, expected 0.764541
2024-05-02 17:21:34.801436: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 1.75582, expected 0.761753
2024-05-02 17:21:34.801465: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 1.30791, expected 0.313837
2024-05-02 17:21:34.801476: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 1.58077, expected 0.586698
2024-05-02 17:21:34.801486: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 1.82882, expected 0.834756
2024-05-02 17:21:34.801497: E external/local_xla

800/800 ━━━━━━━━━━━━━━━━━━━━ 125s 149ms/step - accuracy: 0.5948 - loss: 0.9062 - val_accuracy: 0.5275 - val_loss: 0.7639
Epoch 2/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 113s 141ms/step - accuracy: 0.6455 - loss: 0.6725 - val_accuracy: 0.6825 - val_loss: 0.5939
Epoch 3/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 113s 140ms/step - accuracy: 0.6393 - loss: 0.6712 - val_accuracy: 0.5838 - val_loss: 0.9236
Epoch 4/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 112s 139ms/step - accuracy: 0.6676 - loss: 0.6314 - val_accuracy: 0.5337 - val_loss: 0.7541
Epoch 5/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 113s 140ms/step - accuracy: 0.6448 - loss: 0.6421 - val_accuracy: 0.6562 - val_loss: 0.6662
Epoch 6/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 112s 139ms/step - accuracy: 0.6690 - loss: 0.6269 - val_accuracy: 0.6500 - val_loss: 0.6700
Epoch 7/50
800/800 ━━━━━━━━━━━━━━━━━━━━ 113s 140ms/step - accuracy: 0.6957 - loss: 0.5896 - val_accuracy: 0.5663 - val_loss: 0.6743


In [126]:
def get_test_data():
    test_gens = {}
    for nl in TEST_NOISE_LEVELS:
        test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
        n = str.replace(str(nl), '.', '_')
        
        test_generator = test_datagen.flow_from_directory(
            test_path+n,
            target_size=(224, 224),
            batch_size=4,
            class_mode='binary',
            color_mode="grayscale",
            shuffle=False,
            seed=42
        )

        test_generator.reset()
        test_gens[nl] = test_generator

    return test_gens

In [127]:
noise_data_gens = get_test_data()

Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [128]:
# Define the class names
class_names = ['Hemorrhage', 'No Hemorrhage']

accuracies = []
all_trues = []
all_preds = []

for noise_level in TEST_NOISE_LEVELS:
    y_pred = get_predictions(model, noise_data_gens[noise_level])
    labels = noise_data_gens[noise_level].labels
    accuracy = metrics.accuracy_score(labels, y_pred)
    print(f"Accuracy of model on noise level {noise_level} = {accuracy}")
    accuracies.append(accuracy)
    all_trues.extend(labels)
    all_preds.extend(y_pred)


200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step
Accuracy of model on noise level 0 = 0.68125
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step
Accuracy of model on noise level 0.25 = 0.6825
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step
Accuracy of model on noise level 0.5 = 0.6725
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step
Accuracy of model on noise level 0.75 = 0.68625
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step
Accuracy of model on noise level 1 = 0.68
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step
Accuracy of model on noise level 1.5 = 0.65625


In [131]:
# Get classification report
class_report = get_classification_report(all_trues, all_preds)
# Convert noise level to a valid string for the filename
filename = f"report_{str(NOISE_LEVEL).replace('.', '_')}_level.txt"
# Save classification report
with open(f"/kaggle/working/{filename}", "w") as file:
    file.write(class_report)

Accuracy of model= 0.6764583333333334


In [129]:
# show confusion matrix
confusion_matrix = metrics.confusion_matrix(all_trues, all_preds)
print(confusion_matrix)

[[1528 1004]
 [ 549 1719]]
